In [2]:
import numpy as np

# We will create a function lev that takes in two strings as arguments and returns
#  the edit distance  between them and the optimal sequence of operations to transform them

def lev(str1,str2):
    
    x,y = str1, str2
    len1, len2 = len(x), len(y)
    
    if len1 > len2:
        x,y = y,x
        len1, len2 = len2, len1
        
        # To compute the edit distance, we will comparing a longer string with a shorter string.
        #  This line of code ensures that the variable y will always denote the longer string.       
        
    L=np.zeros(shape=(len1+1,len2+1))
    
    # We will need to do dynamic programming to compute the solution to thi problem. Thus, we first create
    # a matrix of zeros whose values we will replace iteratively as we work through the rows of the table.
    # There are len1+1 rows and len2+1 columns because we will require the case of an empty string for the
    # purposes of dynamic programming.
    
    # The columns represent the characters of string 2, the longer string and the rows represent the 
    # characters of string 1, the shorter string. The index of the column and rows denote the substring up 
    # to its ith character. For example, row 2 denotes the string made up of the first 2 characters of string
    # 1, and column 4 denotes the string made up of the first 4 characters of string 2.
    
    L[0,:]=np.linspace(0,len2, len2+1)
    L[:,0]=np.linspace(0,len1, len1+1)
    
    # The distance between an empty string (no word) and a string of length m will be m, as m deletions are
    # required to get from the string to an empty string. Thus, I can fill up the column 0 and row 0. Each
    # entry with index (i,j) in the matrix represents the edit distance between the string made up of the
    # first i characters of string 1 and the string made up of the first j characters of string 2.

    for i in range(1,len1+1):
        for j in range(1,len2+1):
            L[i,j]=min(L[i-1,j]+1,L[i,j-1]+1,L[i-1,j-1]+np.where(x[i-1]==y[j-1],0,1))
    
    # Here we implement the main portion of the code which will help us to calculate the edit distance. 
    # The edit distance between a string made up of the first i characters of x and the first j characters
    # of y (henceforth referred to as dist(i,j)) can be calculated from the edit distances of the different
    # cases it took to get there.
    
    # If the ith character and the jth character are the same, then dist(i,j) = dist(i-1,j-1) since we do not
    # have to perform an operation on either character. However, if they are not the same, then we need to 
    # perform a substitution to make the ith character of the first string equal to the jth character of the
    # second string. In that case, dist(i,j) = dist(i-1,j-1) + 1.
    
    # There are two other cases. dist(i,j) = dist(i-1,j) + 1 because the string consisting of the first i 
    # characters of x can be made into the string consisting of the first i-1 characters of x by deleting one
    # character. Likewise for the substring of y: dist(i,j) = dist(i,j-1) + 1. Since we want to take the 
    # minimum number of operations, we want to choose the minimum of these cases, which is expressed in the 
    # equation above. In executing the code above, we populate our matrix with the edit distances between 
    # various substrings of x and y and obtain the edit distance between x and y in the bottom right corner 
    # of the matrix.

    path=[]
    
    # This stores the path of optimal sequence
    
    i=len1
    j=len2

        
    # We iterate over the longer string y and decide whether to keep, substitute, or delete a character so 
    # that we end up with the smaller one.
    
    while (i,j)!=(0,0):
        if i==0:
            if j!=0:
                if L[i,j-1] < L[i,j]:
                    j = j-1
                    path.append('delete')
                else:
                    j = j-1
                    path.append('keep')
         
        # When i=0, there is no option to move diagonally upwards anymore since that is the top row. At this
        # point, no further substitution will be required since substitution requires a diagonal move upwards
        # to the left. As we move left across the row, if the edit distance remains the same, we keep the 
        # character and if the edit distance decreases from the jth to the j-1th character, we delete a character
        # since deletion is implied by a decrease in edit distance.
        
        else:
            if L[i-1,j-1]+np.where(x[i-1]==y[j-1],0,1) <= L[i,j-1]+1 and L[i-1,j-1]+np.where(x[i-1]==y[j-1],0,1) <= L[i-1,j]+1:
                i = i-1
                j = j-1
                if x[i]==y[j]:
                    path.append('keep')
                    
                    # when the option of dist(i-1,j-1) results in the minimum edit distance, and  x[i] = y[j], 
                    # then we keep the jth character of y.
                    
                else:
                    path.append('substitute')
                    
                    # when dist(i-1,j-1) is minimum and x[i] != y[j], then we have to substitute the jth 
                    # character of y to fit the ith character of x
                    
            elif L[i,j-1]<L[i-1,j] and L[i,j-1] + 1 < L[i-1,j-1]+np.where(x[i-1]==y[j-1],0,1):
                j = j-1
                path.append('delete')
                
                # since we are going over the longer string y, I only consider moments when the dist(i,j-1) is 
                # minimum so that I either move left or diagonally left upwards from L[len1,len2] to L[0,0]. If
                # dist(i,j-1) is minimum, that means that a deletion of the jth character is required for y.
                

    return L[len1,len2] , path

In [3]:
lev('kitten','sitting')

(3.0, ['delete', 'keep', 'substitute', 'keep', 'keep', 'keep', 'substitute'])

In [12]:
def pen_lev(str1,str2):
    
    x,y = str1, str2
    len1, len2 = len(x), len(y)
    
    if len1 > len2:
        x,y = y,x
        len1, len2 = len2, len1
    
    # We retain most of the code from the lev function. However, we need to make some small changes to 
    # penalize insertion and deletion.
        
    L=np.zeros(shape=(len1+1,len2+1))
    

    L[0,:]=np.linspace(0,2*len2, len2+1)
    L[:,0]=np.linspace(0,2*len1, len1+1)
    
    # Here, we double the number of operations for dist(0,j) and dist(i,0) so that the edit distance is
    # 2*j or 2*i as opposed to just j or i. 

    for i in range(1,len1+1):
        for j in range(1,len2+1):
            L[i,j]=min(L[i-1,j]+2,L[i,j-1]+2,L[i-1,j-1]+np.where(x[i-1]==y[j-1],0,1))
            
            # Here we for dist(i-1,j) and dist(i,j-1), instead of just adding one, we add two since the
            # operation cost is doubled.

    
    # The optimal sequence searcher works in the same way since the penalty only changes the edit distance
    # values and not the way the search works. 
    
    path=[]
    
    i=len1
    j=len2

        
    while (i,j)!=(0,0):
        if i==0:
            if j!=0:
                if L[i,j-1] < L[i,j]:
                    j = j-1
                    path.append('delete')
                else:
                    j = j-1
                    path.append('keep')
            
        else:
            if L[i-1,j-1]+np.where(x[i-1]==y[j-1],0,1) <= L[i,j-1]+1 and L[i-1,j-1]+np.where(x[i-1]==y[j-1],0,1) <= L[i-1,j]+1:
                i = i-1
                j = j-1
                if x[i]==y[j]:
                    path.append('keep')
                else:
                    path.append('substitute')
            elif L[i,j-1]<L[i-1,j] and L[i,j-1] + 1 < L[i-1,j-1]+np.where(x[i-1]==y[j-1],0,1):
                j = j-1
                path.append('delete')
                
    return L[len1,len2], path